In [2]:
from mdp import PendulumMDP
import numpy as np
import scipy as sp
import cvxopt

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
mdp = PendulumMDP()
x0, _ = mdp.reset()
print x0

[ 3.14159265  0.        ]


In [4]:
from algo.optim.ilqg import jacobian, grad, linearize

# We're trying to solve the following optimization problem:
# min  L(X, U) s.t. x_t+1 = f(x_t, u_t)
# (let's ignore other kinds of constraints for now)
# We do this by turning the constraints into costs, and gradually increase the penalty:
# min  L(X, U) + mu*|x_t+1 - f(x_t, u_t)|
# This unconstrained problem is then solved via Sequential Quadratic Programming,
# where we form a quadratic approximation to the cost, and a linear approximation to the
# dynamics. To make sure that the approximation is valid, we impose a trust region constraint
#
# Unlike iLQG, the output of this optimization won't give us a stabilizing controller. We have
# two options:
# - Perform model predictive control
# - feed the output as an input to iLQG to obtain time-varying controllers

In [5]:
# # print xinit.shape
# # print uinit.shape

# sysdyn = mdp.forward_dynamics
# cost_func = mdp.cost
# final_cost_func = mdp.final_cost
# x = np.array(xinit)
# u = np.array(uinit)
# #print fx[0]
# # At each time step, we have: x_t+1 - xhat_t+1 ~ fx.dot(x_t - xhat_t) + fu.dot(u_t - uhat_t)
# # and l(x_t, u_t) - l(xhat_t, uhat_t) ~
# #         cx.dot(x_t - xhat_t) + cu.dot(u_t, uhat_t)
# #      + (x_t - xhat_t).T.dot(cxx).dot(x_t - xhat_t)
# #      + (u_t - uhat_t).T.dot(cuu).dot(u_t - uhat_t)
# #      + 2*(x_t - xhat_t).T.dot(cxu).dot(u_t - uhat_t)
# # We need to formulate this and pass it to a quadratic solver

In [6]:
import cvxpy
#xinit

In [7]:
#np.vstack([np.hstack([cxx[0], cxu[0]]), np.hstack([cxu[0].T, cuu[0]])])

In [8]:
def compute_cost(x, xref, u, merit, cost_func, final_cost_func):
    loss = 0
    N = x.shape[0] - 1
    loss = final_cost_func(x[N])
    for t in range(N):
        loss += cost_func(x[t], u[t])
        loss += merit * np.sum(np.abs(x[t+1] - xref[t+1]))
    #import ipdb; ipdb.set_trace()
    return loss

def compute_violation(x, xref):
    vio = 0
    for t in range(N):
        vio += np.sum(np.abs(x[t+1] - xref[t+1]))
    return vio

In [9]:
# x0, _ = mdp.reset()
# xinit = mdp.xinit
# N = mdp.horizon
# Du = mdp.n_actions
# Dx = len(x0)
# uinit = np.zeros((N, Du))

# sysdyn = mdp.forward_dynamics
# cost_func = mdp.cost
# final_cost_func = mdp.final_cost
# x = np.array(xinit)
# u = np.array(uinit)
# improve_ratio_threshold = 0.25
# trust_box_size = 0.1
# trust_shrink_ratio = 0.1
# trust_expand_ratio = 1.5
# max_merit_itr = 5
# merit_increase_ratio = 10
# min_trust_box_size = 1e-4
# min_model_improve = 1e-4

# merit = 10.0
# for merit_itr in range(max_merit_itr):
#     xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]
#     before_cost = compute_cost(x, xref, u, merit, cost_func, final_cost_func)
#     while True:
#         dx = [cvxpy.Variable(Dx) for _ in range(N+1)]
#         du = [cvxpy.Variable(Du) for _ in range(N)]
#         xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]

        
#         fx, fu, cx, cu, cxx, cxu, cuu = linearize(x, u, sysdyn, cost_func, final_cost_func)
#         #before_cost = 
#         loss = dx[N].T * cx[N] + cvxpy.quad_form(dx[N], cxx[N]) + final_cost_func(x[N])
#         for t in range(N):
#             cquad = np.vstack([np.hstack([cxx[t], cxu[t]]), np.hstack([cxu[t].T, cuu[t]])])
#             #mat = np.con
#             loss += dx[t].T * cx[t] + du[t].T * cu[t] + cvxpy.quad_form(cvxpy.vstack(dx[t], du[t]), cquad) + cost_func(x[t], u[t])
#             loss += merit * cvxpy.norm(x[t+1] + dx[t+1] - xref[t+1] - fx[t] * dx[t] - fu[t] * du[t], 1)
            
#         objective = cvxpy.Minimize(loss)
        
#         no_improve = False
            
#         while trust_box_size > min_trust_box_size:
#             xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]
#             before_cost = compute_cost(x, xref, u, merit, cost_func, final_cost_func)
#             constraints = [cvxpy.abs(cvxpy.vstack(dx[t], du[t])) <= trust_box_size for t in range(N)]
#             constraints += [dx[0] == 0]
#             prob = cvxpy.Problem(objective, constraints)
#             after_cost = prob.solve(max_iters=100)#, verbose=True)#, solver=cvxpy.CVXOPT)
#             #print result
#             unew = np.zeros_like(u)
#             xnew = np.zeros_like(x)
#             for t, dut in enumerate(du):
#                 unew[t] = u[t] + dut.value.reshape(-1)
#             for t, dxt in enumerate(dx):
#                 xnew[t] = x[t] + dxt.value.reshape(-1)
#             model_improve = before_cost - after_cost
#             if model_improve < -1e-5:
#                 print "approximate merit function got worse (%f). (convexification is probably wrong to zeroth order)" % model_improve
#             if model_improve < min_model_improve:
#                 print "converged because improvement was small (%f < %f)" % (model_improve, min_model_improve)
#                 no_improve = True
#                 break
#             xnewref = [None] + [sysdyn(xnew[t], unew[t]) for t in range(N)]
#             true_after_cost = compute_cost(xnew, xnewref, unew, merit, cost_func, final_cost_func)
#             print "cost before: ", before_cost, "cost after: ", after_cost, "true cost after: ", true_after_cost

#             true_improve = before_cost - true_after_cost
#             #import ipdb; ipdb.set_trace()
#             improve_ratio = true_improve / model_improve
#             if improve_ratio >= improve_ratio_threshold:
#                 trust_box_size *= trust_expand_ratio
#                 print "trust box expanded to %f" % trust_box_size
#                 x = xnew
#                 u = unew
#                 xref = xnewref
#                 break
#             else:
#                 trust_box_size *= trust_shrink_ratio
#                 print "trust box shrunk to %f" % trust_box_size
#         if trust_box_size < min_trust_box_size:
#             print "converged because trust region is tiny"
#             break
#         if no_improve:
#             break

#         #before_cost = true_after_cost
        
# #     x = xnew
# #     u = unew
# #     xref = xnewref
#     if compute_violation(x, xref) < 1e-5:
#         print 'all constraints satisfied!'
#         break
#     merit *= merit_increase_ratio

#         #print map(lambda x: x.value, du)

In [13]:
from gurobipy import *

x0, _ = mdp.reset()
xinit = mdp.xinit
N = mdp.horizon
Du = mdp.n_actions
Dx = len(x0)
uinit = np.zeros((N, Du))

sysdyn = mdp.forward_dynamics
cost_func = mdp.cost
final_cost_func = mdp.final_cost
x = np.array(xinit)
u = np.array(uinit)
improve_ratio_threshold = 0.25
trust_box_size = 0.1
trust_shrink_ratio = 0.6
trust_expand_ratio = 1.5
max_merit_itr = 5
merit_increase_ratio = 10
min_trust_box_size = 1e-4
min_model_improve = 1e-4

In [14]:
# a, b are lists of gurobi variables

import operator
def inner_product(a, b):
    return quicksum(map(operator.mul, a, b))

def quad_form(xs, coeff):
    Nx = len(xs)
    expr = QuadExpr()
    expr.addTerms(coeff.reshape(-1), [y for x in xs for y in [x] * Nx], [y for _ in range(Nx) for y in xs])
    return expr

In [15]:
merit = 10.0
for merit_itr in range(max_merit_itr):
    xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]
    before_cost = compute_cost(x, xref, u, merit, cost_func, final_cost_func)
    while True:
        dx = [[None] * Dx for _ in range(N+1)]
        du = [[None] * Du for _ in range(N)]
        model = Model("sqp")
        model.setParam(GRB.param.OutputFlag, 0)
        for t in range(N+1):
            for k in range(Dx):
                dx[t][k] = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name='dx_%d_%d' % (t, k))
        for t in range(N):
            for k in range(Du):
                du[t][k] = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, name='du_%d_%d' % (t, k))
        norm_aux = [[None] * Dx for _ in range(N)]#* N
        for t in range(N):
            for k in range(Dx):
                norm_aux[t][k] = model.addVar(lb=0, ub=GRB.INFINITY, name="norm_aux_%d_%d" % (t, k))
        model.update()
        
        xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]

        fx, fu, cx, cu, cxx, cxu, cuu = linearize(x, u, sysdyn, cost_func, final_cost_func)
        
        loss = inner_product(dx[N], cx[N]) + quad_form(dx[N], cxx[N]) + final_cost_func(x[N])
            
#         #before_cost = 
        
#         loss = dx[N].T * cx[N] + cvxpy.quad_form(dx[N], cxx[N]) + final_cost_func(x[N])
        for t in range(N):
            cquad = np.vstack([np.hstack([cxx[t], cxu[t]]), np.hstack([cxu[t].T, cuu[t]])])
#             #mat = np.con
            loss += inner_product(dx[t], cx[t]) + inner_product(du[t], cu[t]) + quad_form(dx[t] + du[t], cquad) + cost_func(x[t], u[t])
            for k in range(Dx):
                loss += merit * norm_aux[t][k]
                rhs = x[t+1,k] + dx[t+1][k] - xref[t+1][k] - inner_product(fx[t,k], dx[t]) - inner_product(fu[t,k], du[t])
#                 rhs += dx[t+1][k] 
#                 rhs += - float(xref[t+1][k])
            
#                 rhs += - float(fx[t,k]) * dx[t][k]
#                 rhs += - float(fu[t,k]) * du[t][k]#inner_product(fx[t], dx[t]) - inner_product(fu[t] * du[t])
                model.addConstr(norm_aux[t][k] >= rhs)
                model.addConstr(norm_aux[t][k] >= -rhs)
        
        model.setObjective(loss, GRB.MINIMIZE)
        
        
        
        
        trust_constraints = []
        
#         objective = cvxpy.Minimize(loss)
        
        no_improve = False
        
        for k in range(Dx):
            model.addConstr(dx[0][k] == 0)
            
        while trust_box_size > min_trust_box_size:
            xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]
            before_cost = compute_cost(x, xref, u, merit, cost_func, final_cost_func)
            
            for constr in trust_constraints:
                model.remove(constr)
            trust_constraints = []
            
            for t in range(1, N+1):
                for k in range(Dx):
                    trust_constraints.append(model.addConstr(dx[t][k] <= trust_box_size))
                    trust_constraints.append(model.addConstr(dx[t][k] >= -trust_box_size))
            
            for t in range(N):
                for k in range(Du):
                    trust_constraints.append(model.addConstr(du[t][k] <= trust_box_size))
                    trust_constraints.append(model.addConstr(du[t][k] >= -trust_box_size))
            
            model.optimize()
            after_cost = model.objVal
            

            unew = np.zeros_like(u)
            xnew = np.zeros_like(x)
            for t, dut in enumerate(du):
                for k, dutk in enumerate(dut):
                    unew[t][k] = u[t][k] + dutk.x
            for t, dxt in enumerate(dx):
                for k, dxtk in enumerate(dxt): 
                    xnew[t][k] = x[t][k] + dxtk.x
            model_improve = before_cost - after_cost
            if model_improve < -1e-5:
                print "approximate merit function got worse (%f). (convexification is probably wrong to zeroth order)" % model_improve
            if model_improve < min_model_improve:
                print "converged because improvement was small (%f < %f)" % (model_improve, min_model_improve)
                no_improve = True
                break
            xnewref = [None] + [sysdyn(xnew[t], unew[t]) for t in range(N)]
            true_after_cost = compute_cost(xnew, xnewref, unew, merit, cost_func, final_cost_func)
            print "cost before: ", before_cost, "cost after: ", after_cost, "true cost after: ", true_after_cost


            true_improve = before_cost - true_after_cost
#             #import ipdb; ipdb.set_trace()
            improve_ratio = true_improve / model_improve
            if improve_ratio >= improve_ratio_threshold:
                trust_box_size *= trust_expand_ratio
                print "trust box expanded to %f" % trust_box_size
                x = xnew
                u = unew
                xref = xnewref
                break
            else:
                trust_box_size *= trust_shrink_ratio
                print "trust box shrunk to %f" % trust_box_size
        if trust_box_size < min_trust_box_size:
            print "converged because trust region is tiny"
            break
        if no_improve:
            break

#         #before_cost = true_after_cost
        
#     x = xnew
#     u = unew
#     xref = xnewref
    if compute_violation(x, xref) < 1e-5:
        print 'all constraints satisfied!'
        break
    merit *= merit_increase_ratio

        #print map(lambda x: x.value, du)

cost before:  68.8470310984 cost after:  62.5021921353 true cost after:  62.3368735007
trust box expanded to 0.150000
cost before:  62.3368735007 cost after:  54.1771254768 true cost after:  53.9043113063
trust box expanded to 0.225000
cost before:  53.9043113063 cost after:  44.0577088563 true cost after:  43.8036794066
trust box expanded to 0.337500
cost before:  43.8036794066 cost after:  32.9375841794 true cost after:  33.5474576651
trust box expanded to 0.506250
cost before:  33.5474576651 cost after:  26.0438505991 true cost after:  26.9146550313
trust box expanded to 0.759375
cost before:  26.9146550313 cost after:  21.0644247319 true cost after:  24.660074173
trust box expanded to 1.139063
cost before:  24.660074173 cost after:  10.7394257104 true cost after:  26.7261643549
trust box shrunk to 0.683438
cost before:  24.660074173 cost after:  14.0622514644 true cost after:  19.8630317338
trust box expanded to 1.025156
cost before:  19.8630317338 cost after:  6.84979947115 true c

In [ ]:
merit = 10.0
for merit_itr in range(max_merit_itr):
    xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]
    before_cost = compute_cost(x, xref, u, merit, cost_func, final_cost_func)
    while True:
        dx = [cvxpy.Variable(Dx) for _ in range(N+1)]
        du = [cvxpy.Variable(Du) for _ in range(N)]
        xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]

        
        fx, fu, cx, cu, cxx, cxu, cuu = linearize(x, u, sysdyn, cost_func, final_cost_func)
        #before_cost = 
        loss = dx[N].T * cx[N] + cvxpy.quad_form(dx[N], cxx[N]) + final_cost_func(x[N])
        for t in range(N):
            cquad = np.vstack([np.hstack([cxx[t], cxu[t]]), np.hstack([cxu[t].T, cuu[t]])])
            #mat = np.con
            loss += dx[t].T * cx[t] + du[t].T * cu[t] + cvxpy.quad_form(cvxpy.vstack(dx[t], du[t]), cquad) + cost_func(x[t], u[t])
            loss += merit * cvxpy.norm(x[t+1] + dx[t+1] - xref[t+1] - fx[t] * dx[t] - fu[t] * du[t], 1)
            
        objective = cvxpy.Minimize(loss)
        
        no_improve = False
            
        while trust_box_size > min_trust_box_size:
            xref = [None] + [sysdyn(x[t], u[t]) for t in range(N)]
            before_cost = compute_cost(x, xref, u, merit, cost_func, final_cost_func)
            constraints = [cvxpy.abs(cvxpy.vstack(dx[t], du[t])) <= trust_box_size for t in range(N)]
            constraints += [dx[0] == 0]
            prob = cvxpy.Problem(objective, constraints)
            after_cost = prob.solve(max_iters=100)#, verbose=True)#, solver=cvxpy.CVXOPT)
            #print result
            unew = np.zeros_like(u)
            xnew = np.zeros_like(x)
            for t, dut in enumerate(du):
                unew[t] = u[t] + dut.value.reshape(-1)
            for t, dxt in enumerate(dx):
                xnew[t] = x[t] + dxt.value.reshape(-1)
            model_improve = before_cost - after_cost
            if model_improve < -1e-5:
                print "approximate merit function got worse (%f). (convexification is probably wrong to zeroth order)" % model_improve
            if model_improve < min_model_improve:
                print "converged because improvement was small (%f < %f)" % (model_improve, min_model_improve)
                no_improve = True
                break
            xnewref = [None] + [sysdyn(xnew[t], unew[t]) for t in range(N)]
            true_after_cost = compute_cost(xnew, xnewref, unew, merit, cost_func, final_cost_func)
            print "cost before: ", before_cost, "cost after: ", after_cost, "true cost after: ", true_after_cost

            true_improve = before_cost - true_after_cost
            #import ipdb; ipdb.set_trace()
            improve_ratio = true_improve / model_improve
            if improve_ratio >= improve_ratio_threshold:
                trust_box_size *= trust_expand_ratio
                print "trust box expanded to %f" % trust_box_size
                x = xnew
                u = unew
                xref = xnewref
                break
            else:
                trust_box_size *= trust_shrink_ratio
                print "trust box shrunk to %f" % trust_box_size
        if trust_box_size < min_trust_box_size:
            print "converged because trust region is tiny"
            break
        if no_improve:
            break

        #before_cost = true_after_cost
        
#     x = xnew
#     u = unew
#     xref = xnewref
    if compute_violation(x, xref) < 1e-5:
        print 'all constraints satisfied!'
        break
    merit *= merit_increase_ratio

        #print map(lambda x: x.value, du)